# Feature Matching
This notebook contains the code to perform feature matching

## Importing libraries

In [1]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import networkx as nx
from IPython.display import Audio, display
import ipynb.fs.defs.Utils as Utils

## Classes definition

In [2]:
#This class contains the methods and properties related to the features of an image
class ImageFeature:
    def __init__(self, image, index):
        self.image = image
        self.index = index  
        
    #Compute the salient points of the image using SIFT algorithm
    def SIFT(self,
             save_output = False, #If True saves the output
             output_dir = "output", #Output directory
             noise_type = None,
             noise_std = 0.0
            ):
        sift = cv.SIFT_create()
        self.kp, self.des = sift.detectAndCompute(self.image,None)
        if noise_type == Utils.NoiseType.POINTS:
            for i in range(len(self.kp)):
                self.kp[i].pt += np.random.normal(scale = noise_std , size = (2,))
        self.img_with_sift=cv.drawKeypoints(self.image,self.kp,np.copy(self.image))
        if save_output:
            cv.imwrite(os.path.join(output_dir,f"{self.index+1}.jpg"), self.img_with_sift)
            

In [3]:
#This class contains the methods and properties related to the problem of feature matching between two images
class Match:
    def __init__(self,
                 image_feature_source, #Source image
                 image_feature_destination #Destination image
                ):
        self.image_feature_source = image_feature_source
        self.image_feature_destination = image_feature_destination
    
    #This function performs feature matching between to images
    def feature_matching(self, threshold, save_output = False, output_dir = "output"):
        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)
        flann = cv.FlannBasedMatcher(index_params,search_params)
        
        bf = cv.BFMatcher()
        matches = bf.knnMatch(self.image_feature_source.des,
                                 self.image_feature_destination.des,
                                 k=2)
        # Need to draw only good matches, so create a mask
        self.good = []
        for m in matches:
            if m[0].distance < threshold*m[1].distance:
                self.good.append((m[0],m[0]))
        draw_params = dict(matchColor = (0,255,0),
                           singlePointColor = (255,0,0),
                           flags = cv.DrawMatchesFlags_DEFAULT)
        self.img_with_match = cv.drawMatchesKnn(self.image_feature_source.image,
                                                self.image_feature_source.kp,
                                                self.image_feature_destination.image,
                                                self.image_feature_destination.kp,
                                                self.good,None,**draw_params)
        
        self.num_matches = len(self.good)
        
        #Compute source and destination points
        self.src_pts = np.double([self.image_feature_source.kp[m[0].queryIdx].pt for m in self.good])
        self.dst_pts = np.double([self.image_feature_destination.kp[m[0].trainIdx].pt for m in self.good])
        
        #Save the output if required
        if save_output:
            cv.imwrite(os.path.join(output_dir,f"match_{self.image_feature_source.index+1}_{ self.image_feature_destination.index+1}.jpg"),self.img_with_match)
    
    #This function allows to check whether the match between the source and destination images is to be considered a good match
    def check_salient(self, threshold, save_output = False, output_dir = "output"):
        #A threshold is used to define the goodness of the  match
        if self.num_matches > threshold:
            if save_output:
                cv.imwrite(os.path.join(output_dir,f"salient_{self.image_feature_source.index+1}_{ self.image_feature_destination.index+1}.jpg"),self.img_with_match)
            return True
        return False
    
    #This function allows to compute and normalize the projected points
    def mult_and_norm(self,H, points):
        proj_p = np.dot(H,points)
        proj_p = proj_p / proj_p[2,:]
        return proj_p[0:2,:]
    
    #This function allows to estimate the homography between the two images
    def fit_homography(self,
                       RANSACmaxIters = 2000, #Number of iterations of RANSAC
                       T_norm = np.eye(3), #Normalization matrix
                       noise_type = Utils.NoiseType.NO_NOISE, #If True introduces some noise in the matches
                       noise_std = 0.1  #Standare deviation of the Gaussian noise
                      ):
        
        
        self.src_pts = np.concatenate((self.src_pts, np.ones([self.src_pts.shape[0],1])), axis=1)
        self.dst_pts = np.concatenate((self.dst_pts, np.ones([self.dst_pts.shape[0],1])), axis=1)
        
        src_pts_proj = self.mult_and_norm(T_norm,self.src_pts.transpose()).transpose() #Compute normalized source points
        dst_pts_proj = self.mult_and_norm(T_norm,self.dst_pts.transpose()).transpose() #Compute normalized destination points
        
        #Compute the homography from source image to destination image
        self.M, _ = cv.findHomography(src_pts_proj, dst_pts_proj, cv.RANSAC, 3*T_norm[0][0], 3.0, maxIters=RANSACmaxIters )

        #If required add some noise to the image
        if noise_type == Utils.NoiseType.HOMOGRAPHY:
            self.M += np.random.normal(scale = noise_std , size = [3,3]) #Add some random noise to the homography
    
    #This function allows to normalize the homography so that the determinant is unitary
    #This makes homographies part of the SL(3) group
    def normalize_homography(self):
        det = np.linalg.det(self.M)
        self.H = self.M/np.cbrt(det)
        new_det = np.linalg.det(self.H)
        
    def copy(self):
        copy = Match(self.image_feature_source,
                    self.image_feature_destination )
        copy.img_with_match = self.img_with_match
        copy.good = self.good
        copy.num_matches = self.num_matches
        copy.src_pts = np.copy(self.src_pts)
        copy.dst_pts = np.copy(self.dst_pts)
        return copy
    
    def get_inv_match(self):
        inv_match = Match(self.image_feature_destination, self.image_feature_source)
        #temp_good = self.good.copy()
        #temp_trainIdx = [m[0].trainIdx for m in temp_good]
        #for i,m in enumerate(self.good):
        #    m[0].trainIdx = m[0].queryIdx
        #    m[0].queryIdx = temp_trainIdx[i]
        #inv_match.good = temp_good
        inv_match.num_matches = self.num_matches
        inv_match.dst_pts = np.copy(self.src_pts)
        inv_match.src_pts = np.copy(self.dst_pts)
        return inv_match
    #Remember: if src is image I and dest is image J, we are finding H from I to J, this is called Hj,i (TODO)
    

## Functions definition

In [4]:
#TODO delete
def allDone():
    print(" --- finished --- ")
    #display(Audio(url='https://sound.peal.io/ps/audios/000/001/131/original/kon.wav', autoplay=True))

In [5]:
#This function allows to compute matches between two images
def compute_matches(imgs, #Images for which matches should be computed
                    T_norm, #Normalization matrix
                    save_images = False, #If True saves the intermediate results
                    matching_threshold = 0.6, #Threshold used to compute matches
                    matches_dir = "matches", #Directory where to save matches
                    matches_th = 20, #Threshold used to compute good matches
                    number_of_matches=1, #Number of times the matching procedure should be repeated
                    noise_std = 0.1, #Standard deviation of the gaussian noise
                    salient_matches_dir = "salient_matches", #Directory where to save good matches
                    sift_dir = "sift", #Directory where to save SIFT results
                    RANSACmaxIters = 2000, #Number of RANSAC iterations
                    noise_type = Utils.NoiseType.NO_NOISE #If True allows to introduce some noise in the homographies
                   ):
    n = len(imgs)
    image_features =[] #Array that will contain the salient features of each image
    
    #Compute salient points of each image using SIFT
    for i in range(0,n):
        for k in range(0,number_of_matches):
            image_feature = ImageFeature(imgs[i], i)
            image_feature.SIFT(save_images, sift_dir, noise_type = noise_type, noise_std = noise_std)
            if len(image_features) <= i:
                image_features.append([image_feature])
            else:
                image_features[i].append(image_feature)
    #Compute the good matches between each pair of images
    matches =[]
    for k in range(0,number_of_matches):
        for i in range(0, n-1):
            for j in range(i+1, n):
                match = Match(image_features[i][k], image_features[j][k])
                match.feature_matching(matching_threshold, save_images, matches_dir)            
                matches.append(match)
    
    salient_matches = list(filter(lambda x: x.check_salient(matches_th, save_images, salient_matches_dir),matches))
    
    #For every good match, compute also the matches in the reverse order (destination, source) and compute homographies
    total_matches = []
    for match in salient_matches:
        inv_match = match.get_inv_match()#Match(match.image_feature_destination, match.image_feature_source)
        #inv_match.feature_matching(matching_threshold, save_images, matches_dir)
        
        match.fit_homography(RANSACmaxIters, T_norm, noise_type = noise_type, noise_std = noise_std)
        match.normalize_homography()
        total_matches.append(match)
        
        inv_match.fit_homography(RANSACmaxIters, T_norm, noise_type = noise_type, noise_std = noise_std)
        inv_match.normalize_homography()
        total_matches.append(inv_match)
        #print(np.linalg.norm(np.eye(3)-match.M@inv_match.M))
    return total_matches

In [6]:
#This function allows to perform the whole feature matching procedure
def get_feature_matches(dataset_name, #Name of the dataset to be used
                imgs, #Images used to compute matches
                T_norm, #Normalization matrix
                matching_threshold = 0.6, #Treshold used to compute matches
                number_of_matches = 1, #Number of times the matching procedure should be repeated
                noise_std = 0.1, #Standard deviation of the Gaussian noise
                matches_th = 20, #Treshold used to compute good matches
                RANSACmaxIters = 2000, #Number of RANSAC iterations
                save_images = True, #If True saves intermediate results
                save_output = True, #If True saves output
                output_dir ="output", #Output directory
                results_dir = "results", #Intermediate results directory
                noise_type = Utils.NoiseType.NO_NOISE, #If True add some Gaussian noise to homographies
                verbose = True #If True prints intermediate results
               ):
        
        #Define directories
        sift_dir = os.path.join(results_dir,'sift')
        matches_dir = os.path.join(results_dir,'matches')
        salient_matches_dir = os.path.join(results_dir,'salient_matches')
        Weight_filename = f"W_{dataset_name}.npy"
        
        #Create directories if not exist
        if save_images:
            shutil.rmtree(results_dir, ignore_errors = True)
            if not os.path.isdir(results_dir):   
                os.mkdir(results_dir)
            if not os.path.isdir(sift_dir):
                os.mkdir(sift_dir)
            if not os.path.isdir(matches_dir):
                os.mkdir(matches_dir)
            if not os.path.isdir(salient_matches_dir):
                os.mkdir(salient_matches_dir)   
            
        n = len(imgs)

        #Compute matches
        matches = compute_matches(imgs = imgs, 
                                  T_norm = T_norm,
                                  save_images= save_images, 
                                  matching_threshold = matching_threshold, 
                                  matches_dir = matches_dir, 
                                  matches_th = matches_th,
                                  number_of_matches = number_of_matches,
                                  noise_std = noise_std,
                                  salient_matches_dir = salient_matches_dir, 
                                  sift_dir = sift_dir,
                                  RANSACmaxIters = RANSACmaxIters,
                                  noise_type = noise_type
                                 )
        
        #Create matches dictionary and weight matrix
        matches_dict = dict()
        weight_matrix = np.zeros([n,n])
        for match in matches:
            i,j = match.image_feature_source.index, match.image_feature_destination.index
            if (i, j) not in matches_dict:
                matches_dict[i, j] = list()
            matches_dict[i, j].append(match.H)
            weight_matrix[i, j] = match.num_matches
        
        #If required save output
        if save_output:
            if not os.path.isdir(output_dir):
                os.makedirs(output_dir)
            np.save(os.path.join(output_dir,Weight_filename), weight_matrix)

        if verbose:#TODO remove
            allDone()
        
        return matches_dict, weight_matrix
        